# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

---

- LLMs are misinterpreted to be a knowledge store
- LLMs are more like "reasoning engines"
- It makes used of the knowledge learnt but can build answer to new questions with a reasoning

---

# Initial Setup


In [1]:
# API Key setup
import os

api = !cat ../`ls -a ../ | grep "gemini"`

api = api[0]

os.environ['GOOGLE_API_KEY'] = api
# universal variable, used by langchain

# Agents

- Agents in LangChain are systems that use a language model to interact with other tools. They can be used for tasks such as grounded question/answering, interacting with APIs, or taking action.

In [12]:
# pip install langchain_experimental

In [13]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain_google_genai import ChatGoogleGenerativeAI

In [26]:
# We will use the LLM as a reasoning engine
# So, temperature=0 is a must, here
llm = ChatGoogleGenerativeAI(model='gemini-pro', 
                             temperature=0.0, 
                             convert_system_message_to_human=True)

In [27]:
# Loading the tools for different tasks
# llm-math is a chain: Uses LLM in conjunction with a calculator
# wikipedia is a an API tool
tools = load_tools(['llm-math', 'wikipedia'], llm=llm)

In [28]:
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.

In [29]:
# Initialize an agent with the tools, llm and agent type
# The agent type here is:
# A zero shot agent that does a reasoning step before acting.
# This agent is designed to be used in conjunction

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True, 
    verbose = True
)

# handle_parsing_errors is activated when the output is not in the 
# desired format. The agent would send it back to the LLM to get abck
# the correct output format.

# LLM-Math Tool

In [30]:
agent.invoke("what is 25% of 300?")



> Entering new AgentExecutor chain...
Question: what is 25% of 300?
Thought: I should multiply 300 by 25%
Action:
```
{
  "action": "Calculator",
  "action_input": "300 * 25 / 100"
}
```

Observation: Answer: 75.0
Thought:Final Answer: 75

> Finished chain.


{'input': 'what is 25% of 300?', 'output': '75'}

## Understanding the output verbose

- First, there is a thought process. what should the LLM do?
- Then, there is an action. This action is a json block 
    - 2 keys: action and action_input
    - Tells what action needs to be performed
    - And what is the input to that action?
- Then, there's an observation with an answer
    - This comes from the calculator tool
- Finally, before the chain ends, we have a final thought
    - This contains the final answer and then closes the chain
    

## Trying out different problems

In [32]:
agent.invoke("what is two plus two equals to? answer in words")



> Entering new AgentExecutor chain...
Question: what is two plus two equals to? answer in words
Thought: I should use the calculator to add two and two
Action:
```
{
  "action": "Calculator",
  "action_input": "2 + 2"
}
```

Observation: Answer: 4
Thought:I should now answer in words
Final Answer: four

> Finished chain.


{'input': 'what is two plus two equals to? answer in words', 'output': 'four'}

In [34]:
agent.invoke("what is two plus two equals to? \
Take the result and then raise it to the power two. \
Answer in words")



> Entering new AgentExecutor chain...
Question: what is two plus two equals to? Take the result and then raise it to the power two. Answer in words
Thought: I should first add two and two, and then square the result
Action:
```
{
  "action": "Calculator",
  "action_input": "2 + 2"
}
```

Observation: Answer: 4
Thought:I should now square the result
Action:
```
{
  "action": "Calculator",
  "action_input": "4 * 4"
}
```


Observation: Answer: 16
Thought:I now know the final answer
Final Answer: sixteen

> Finished chain.


{'input': 'what is two plus two equals to? Take the result and then raise it to the power two. Answer in words',
 'output': 'sixteen'}

# Wikipedia API
- Wikipedia API is glitchy with, yet again, Google's APIs
- So, some queries do not run fully

In [50]:
question = "What is the India?"

In [51]:
result = agent(question)



> Entering new AgentExecutor chain...
Question: What is the India?
Thought: I should search for India on Wikipedia
Action:
```
{
  "action": "wikipedia",
  "action_input": "India"
}
```

Observation: Page: India
Summary: India, officially the Republic of India (ISO: Bhārat Gaṇarājya), is a country in South Asia.  It is the seventh-largest country by area; the most populous country as of June 2023; and from the time of its independence in 1947, the world's most populous democracy. Bounded by the Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the southeast, it shares land borders with Pakistan to the west; China, Nepal, and Bhutan to the north; and Bangladesh and Myanmar to the east. In the Indian Ocean, India is in the vicinity of Sri Lanka and the Maldives; its Andaman and Nicobar Islands share a maritime border with Thailand, Myanmar, and Indonesia.
Modern humans arrived on the Indian subcontinent from Africa no later than 55,000 years ago.
The

In [57]:
result = agent.invoke("What is the latest MacOS version name?")



> Entering new AgentExecutor chain...
Thought: I can use wikipedia to find the latest MacOS version name
Action:
```
{
  "action": "wikipedia",
  "action_input": {
    "search": "macOS"
  }
}
```


TypeError: WikipediaQueryRun._run() got an unexpected keyword argument 'search'

# Coding using LLM
- Write code using LLM and then run it as well

In [60]:
repl = PythonREPLTool()

In [70]:
print(repl.invoke("```python \
for i in range(5):\n \
    print(i)\n \
```"))

0
1
2
3
4



In [71]:
agent = create_python_agent(
    llm, 
    tool = PythonREPLTool(),
    verbose = True
)

In [72]:
cust_list = [
    ['Pushpak', 'Ruhil'],
    ['Bharti', 'Ruhil'],
    ['Aman', 'Antony'],
    ['Zain', 'Antony'],
    ['Chinmay', 'Tushir'],
    ['Anoop', 'Udayan']
]

In [80]:
agent.invoke(f"""sort the customers by \
the last name in ascending order and then the \
first name and print the output \

customer_list = {cust_list}
""")




> Entering new AgentExecutor chain...
Action: Python_REPL
Action Input: print(sorted(customer_list, key=lambda x: (x[1], x[0])))
Observation: [['Aman', 'Antony'], ['Zain', 'Antony'], ['Bharti', 'Ruhil'], ['Pushpak', 'Ruhil'], ['Chinmay', 'Tushir'], ['Anoop', 'Udayan']]

Thought:Final Answer: [['Aman', 'Antony'], ['Zain', 'Antony'], ['Bharti', 'Ruhil'], ['Pushpak', 'Ruhil'], ['Chinmay', 'Tushir'], ['Anoop', 'Udayan']]

> Finished chain.


{'input': "sort the customers by the last name in ascending order and then the first name and print the output \ncustomer_list = [['Pushpak', 'Ruhil'], ['Bharti', 'Ruhil'], ['Aman', 'Antony'], ['Zain', 'Antony'], ['Chinmay', 'Tushir'], ['Anoop', 'Udayan']]\n",
 'output': "[['Aman', 'Antony'], ['Zain', 'Antony'], ['Bharti', 'Ruhil'], ['Pushpak', 'Ruhil'], ['Chinmay', 'Tushir'], ['Anoop', 'Udayan']]"}

# Defining Our Own Tools

In [83]:
from langchain.agents import tool
from datetime import date

# Doc string required for @tool

@tool # Using a tool decorator to wrap the time function we define
def time(text: str) -> str:
    """Returns todays date. 
    Use this for any question related to knowing today's date.
    The input should always be an empty string and
    this function will always return todays date - 
    any date mathematics should occur outside this function
    """
    return str(date.today()) 

In [88]:
agent = initialize_agent(
    tools + [time],
    llm,
    agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True,
    verbose = True
)

In [92]:
agent.invoke("What is today's date?")



> Entering new AgentExecutor chain...
Thought: I can use the time tool to get today's date
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2024-03-30
Thought:Final Answer: 2024-03-30

> Finished chain.


{'input': "What is today's date?", 'output': '2024-03-30'}